# Предобработка датасета с ФИО на кириллице

In [1]:
import pandas as pd
import re
import json

In [2]:
# Импортируем data.csv и преобразуем в pandas dataframe
df = pd.read_csv("data.csv", header=None)
# Жестко удаляем все NaN и прочее
df = df.dropna()

In [3]:
# Необходима минимальная предобработка:
# Регулярное выражение предобработки данных ФИО согласно требованиям законодательства РФ
regexp_fio = r"[^а-яА-ЯёЁ'-.,IV()\s]"
# Регулярное выражение предобработки данных Gender, где должны остаться только М и Ж
regexp_gender = r"[^МЖ]"
# Регулярное выражение предобработки данных для удаления *
regexp_star = r"[*]"

In [4]:
### OPTIONAL: проверка алгоритма чистки данных
# import pandas as pd
# df_test = pd.DataFrame({
#     'Фамилия': ['Ив1анов', 'П***етров', 'Си_доров'],
#     'Имя': ['Иван', 'Петр', 'Сергей'],
#     'Отчество': ['Иванович', 'Петрович', 'Сергеевич'],
#     'Пол': ['М', 'М', 'М']
# })

# df_test['Фамилия'] = df_test['Фамилия'].str.replace(r"[^а-яА-ЯёЁ'-.,IV()\s]", '', regex=True).replace(r"[*]", '', regex=True)
# df_test.head()

In [5]:
# Применение алгоритма предобработки к датасету
df[0] = df[0].str.replace(regexp_fio, '', regex=True).replace(regexp_star, '', regex=True)
df[1] = df[1].str.replace(regexp_fio, '', regex=True).replace(regexp_star, '', regex=True)
df[2] = df[2].str.replace(regexp_fio, '', regex=True).replace(regexp_star, '', regex=True)
df[3] = df[3].str.replace(regexp_gender, '', regex=True).replace(regexp_star, '', regex=True)

In [6]:
# Убираем нулевой Gender, в России существует только 2 гендера
display(df[3].value_counts())
df = df[(df[3] == 'М') | (df[3] == 'Ж')]

Ж    3503762
М    2721944
           8
Name: 3, dtype: int64

##### Алгоритм предобработки ФИО на примере фамилии, что также будет применено и к ИО

In [7]:
# Группируем данные по фамилии и считаем количество встречаемости каждой фамилии
grouped = df.groupby([0, 3])
counts = grouped[0].count()
# Создаем новый DataFrame с результатами
result_df = pd.DataFrame({'Фамилия': grouped.groups, 'Количество': counts})
# Убираем столбец с типами данных столбца Фамилия
result_df = result_df.drop('Фамилия', axis=1)
# Выводим полученный DataFrame
result_df.head(10)

# Дальнейшими предобработками предлагаю пренебречь, так как реализуемый далее алгоритм
# исправления ошибок в ФИО очень маловероятно на них наткнется

Количество
          Ж          82
          М           5
-         Ж           4
          М           1
---       Ж           1
-БАЛАКИНА Ж           1
-СОКОЛОВ  М           1
.ВАНЮШКИН М           1
.ИЛЮШКИНА Ж           1
.КРЮЧКОВ  М           1

In [8]:
# Именно в таком виде значения построчно хранятся в файлах .jsonl, столбец количество - показатель релевантности ФИО
result_df = result_df.sort_values("Количество", ascending=False)

In [9]:
# Превращаем в словарь для дальнейшей работы с полученными данными
result_dict = result_df.to_dict()

Получаем словарь вида:

```
{'Количество': {('ИВАНОВА', 'Ж'): 24238,
  ('КУЗНЕЦОВА', 'Ж'): 21113,
  ('ИВАНОВ', 'М'): 18492,
  ...}
}
```

In [10]:
# Запись в файл формата .jsonl, где структура единичного элемента:
# {"name": str(), "gender": М/Ж, "relevance": int()}
with open('surname_data.jsonl', 'w', encoding='utf8') as file:
    for key, value in result_dict["Количество"].items():
        item = {
            "name": key[0],
            "gender": key[1],
            "relevance": value
        }
        file.write(json.dumps(item, ensure_ascii=False) + '\n')

##### Алгоритм в виде функции

In [11]:
def extract_fio_data_to_jsonl(dataframe, column, filename):
    grouped = dataframe.groupby([column, 3])
    counts = grouped[column].count()
    result_dataframe = pd.DataFrame({'Фамилия': grouped.groups, 'Количество': counts})
    result_dataframe = result_dataframe.drop('Фамилия', axis=1)
    result_dataframe = result_dataframe.sort_values("Количество", ascending=False)
    result_records = result_dataframe.to_dict()
    with open(f'{filename}.jsonl', 'w', encoding='utf8') as file:
        for key, value in result_records["Количество"].items():
            item = {
                "name": key[0],
                "gender": key[1],
                "relevance": value
            }
            file.write(json.dumps(item, ensure_ascii=False) + '\n')

In [12]:
extract_fio_data_to_jsonl(df, 0, "surname_data")
extract_fio_data_to_jsonl(df, 1, "name_data")
extract_fio_data_to_jsonl(df, 2, "middlename_data")